# Julia for Global Health Research and Collaboration

## 2023 OHDSI DevCon

### Jacob S. Zelko

# My Name Is Jacob Zelko!

- TheCedarPrince; Open Source Developer (Javis.jl, JuliaHealth, the cedar ledge)

- Chair of Julia Community Development Fund & Mentor for Julia Gender Inclusive

- Researcher @ GTRI and CDC

- Applied Math MS @ NEU (Roux Institute)

![](resized_about.jpeg)

# What Are the Goals of OHDSI?

1. **International open science collaborative dedicated to observational health research**

2. Establish open community data standards

3. **Make end to end solutions of translating observational data to reliable evidence**

# What Am I Doing in OHDSI?

Develop package to characterize populations with mental health conditions, investigate prevalence of mental health care, and utilization of mental health resources in rural and urban communities.

1. Identify vulnerable populations and their characteristics

2. Examine loss to follow up

3. Conduct large scale observational health research

# Where Is the Project Today?

- Deploying study with collaborators across the globe

- Final weeks of ongoing study!

- Stretch Goal: 100 million patients

# What Is Julia?

![](julia_gif.gif)

_Walks like Python, talks like Lisp, runs like Fortran_

## Brief History

- Created at MIT in 2012

- Designed for scientific, numerical and high performance computing

- Addresses the "Two Language" problem

## Two Language Problem?

- You start out prototyping in one language

- Performance needs forces switch to a different one

Julia is supposed to be fast so let's put it to the test:

In [4]:
"""
Naive implementation of sum. Works for any iterable `x` with any element type.
"""
function my_sum(x)
    result = zero(eltype(x))
    for element in x
        result += element
    end
    return result
end

my_sum

Let's make some fake data to test `my_sum` with:

In [9]:
data = rand(Float64, 10^3)
data = data .* 10 |> result -> round.(Int, result);

And finally let's time `my_sum`:

In [10]:
using BenchmarkTools
@btime my_sum($data)

  67.112 ns (0 allocations: 0 bytes)


5112

Hm, can we make it faster?

In [6]:
function my_fast_sum(x)
    result = zero(eltype(x))
    @inbounds @simd for element in x
        result += element
    end
    result
end

my_fast_sum (generic function with 1 method)

Let's time it:

In [11]:
@btime my_fast_sum($data)

  44.492 ns (0 allocations: 0 bytes)


5112

Great! Let's see how these all compare starting with our original `my_sum`:

In [14]:
@btime my_sum($data)

  67.110 ns (0 allocations: 0 bytes)


5112

Then `my_fast_sum`

In [13]:
@btime my_fast_sum($data)

  44.487 ns (0 allocations: 0 bytes)


5112

And let's see how it compares to the native Julia `sum` function

In [12]:
@btime sum($data)

  65.855 ns (0 allocations: 0 bytes)


5112

# Building Bridges with Julia
![](bridges.jpg)

# Building Bridges with Julia

![](jacob.jpg)

![](hadley.jpg)

# Patient Characterization in Julia
- Goal: Create characterization of patients across three axes who have had at least one prior diagnosis of strep throat.
- Concept Set: $28060$ (Streptococcal sore throat; SNOMED)

## Environment Set-Up

For all Julia projects, you only need a Project.toml and Manifest.toml.
These specify package dependencies and their versions down to transient dependencies.

In [2]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Errata/DevCon`


We'll be using the following packages:

- [`DataFrames`](https://github.com/JuliaData/DataFrames.jl) - Julia's dataframe handler for easily manipulating data

- [`OMOPCDMCohortCreator`](https://github.com/JuliaHealth/OMOPCDMCohortCreator.jl) - Iteratively create patient cohorts from databases utilizing the OMOP CDM

- [`HealthSampleData`](https://github.com/JuliaHealth/HealthSampleData.jl) - Sample health data for a variety of health formats and use cases

- [`SQLite`](https://github.com/JuliaDatabases/SQLite.jl) - A Julia interface to the SQLite library

# Data

We'll use data provided from Eunomia that is stored in a SQLite format.

In [ ]:
import HealthSampleData: Eunomia
eunomia = Eunomia();

To connect to the database, we'll do the following:

In [ ]:
import SQLite: DB
conn = DB(eunomia);

To generate connection details, we do the following (NOTE: this is boiler plate that will eventually be handled in the background):

In [ ]:
import OMOPCDMCohortCreator as occ

occ.GenerateDatabaseDetails(:sqlite, "");
occ.GenerateTables(conn);

## Finding all Patients with Strep Throat

In [ ]:
strep_patients = occ.ConditionFilterPersonIDs(28060, conn);
size(strep_patients) |> num -> println("Number of patients with Strep: $(num[1])")

## Finding race of all Patients with Strep Throat

In [ ]:
strep_patients_race = occ.GetPatientRace(strep_patients.person_id, conn)
strep_patients_race[1, :] |> println

## Finding gender of all Patients with Strep Throat

In [ ]:
strep_patients_gender = occ.GetPatientGender(strep_patients.person_id, conn)
strep_patients_gender[1, :] |> println

## Creating age groupings of all Patients with Strep Throat

In [ ]:
age_groups = [[0, 4], [5, 9], [10, 14],	[15, 19], [20, 24], [25, 29], [30, 34],
[35, 39], [40, 44], [45, 49], [50, 54], [55, 59], [60, 64], [65, 69], [70, 74],
[75, 79], [80, 84], [85, 89], [90, 94], [95, 99], [100, 104], [105, 109]]
strep_patients_age_group = occ.GetPatientAgeGroup(strep_patients.person_id, conn; age_groupings = age_groups)
strep_patients_age_group[1, :] |> println

Aside, we can generate SQL representations of all queries used so far

In [ ]:
occ.GetPatientAgeGroup([1]; age_groupings = age_groups) |> println

## Characterize Each Person by Gender, Race, and Age Group

In [ ]:
import DataFrames as DF

strep_patients_characterized = DF.outerjoin(strep_patients_race, strep_patients_gender, strep_patients_age_group; on = :person_id, matchmissing = :equal)
strep_patients_characterized[1, :] |> println

# Finalizing Patient Characterization
We can now group up our patients into population groups

In [ ]:
strep_patients_characterized = strep_patients_characterized[:, DF.Not(:person_id)]

strep_patient_groups = DF.groupby(strep_patients_characterized, [:race_concept_id,
:gender_concept_id,
:age_group]
)

strep_patient_groups = DF.combine(strep_patient_groups, DF.nrow => :count)

## Execute Safety Audit

In [ ]:
audited_strep_patient_groups = occ.ExecuteAudit(strep_patient_groups; hitech = true)

# Ways To Get Involved

- Experiment with Julia Health packages and ecosystem
  - Patient Level Prediction
  - Coarse Treatment Pathways
  - HPC ETL Pipeline

- Drop into JuliaHealth Meetings or Julia Community

- Running Birds of a Feather event at JuliaCon (Boston)


# Let's Do Network Studies!

- Conducting network studies is entirely possible in Julia

- Growing network of sites are friendly to Julia projects
  - Georgia Tech Research Institute
  - Brown University DBMI
  - Tufts CTSI

- Enable more people to participate

# Concluding Thoughts

![](alan.jpg)

# Keep in Touch!

- Email: jacob.zelko@gtri.gatech.edu

- Website: https://jacobzelko.com/resources/

- Twitch: https://www.twitch.tv/thecedarprince

- OHDSI Microsoft Teams

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*